# Inversion Steps:
1. Setup
    * Import modules
    * Download/format data
        * Data
        * Times for data
    * Load data into variables
2. Specify source/receiver parameters
    * Location (source/receiver)
    * Orientation (source/receiver)
    * Peak current amplitude (source)
    * Radius (source)
    * Phenomenon to measure (receiver)
3. Create Model Cells/mesh (1D)
    * Since this is a 1D model, it is really just depths/thicknesses of the "layers" or "cells" we will be using
4. Define a starting model (0.1 Ohm meters (or S/m?))
    * Make mesh be natural log values (since large differences in potential values)
5. Generate 1D Simulation of data based on survey parameters
6. Set up inversion parameters
    * Weight our data (to help calculate least-squares misfit)
    * Define regularization/model objective function
    * Create a reference model
    * Define sparse and blocky norms
    * Define how optimization is solved (Gauss-Newton with conjugate gradient solver)
    * Define the inverse problem

# Import modules

In [ ]:
import os
import tarfile
import numpy as np
import matplotlib.pyplot as plt

from discretize import TensorMesh

import simpeg.electromagnetics.time_domain as tdem

from simpeg.utils import mkvc, plot_1d_layer_model
from simpeg import (
    maps,
    data,
    data_misfit,
    inverse_problem,
    regularization,
    optimization,
    directives,
    inversion,
    utils,
)

plt.rcParams.update({"font.size": 16, "lines.linewidth": 2, "lines.markersize": 8})


# Download and format data

In [ ]:
import pathlib
emDataDir = pathlib.Path("./EMSampleData").absolute()
data_filename = emDataDir.joinpath("em1dtm_data.txt")
print(data_filename)

# Load data

In [ ]:
# Load field data
dobsIN = np.loadtxt(str(data_filename), skiprows=1)
dobsIN

In [ ]:
# Break down data into two variables: time and data
times = dobsIN[:, 0]
dobs = mkvc(dobsIN[:, -1])

fig = plt.figure(figsize=(7, 7))
ax = fig.add_axes([0.15, 0.15, 0.8, 0.75])
ax.loglog(times, np.abs(dobs), "k-o", lw=3)
ax.set_xlabel("Times (s)")
ax.set_ylabel("|B| (T)")
ax.set_title("Observed Data")

# Create Digital Equipment/Source model

In [ ]:
# Source loop geometry
source_location = np.array([0.0, 0.0, 20.0])
source_orientation = "z"  # "x", "y" or "z"
source_current = 1.0  # peak current amplitude
source_radius = 6.0  # loop radius

# Receiver geometry
receiver_location = np.array([0.0, 0.0, 20.0])
receiver_orientation = "z"  # "x", "y" or "z"

# Receiver list
receiver_list = []
receiver_list.append(
    tdem.receivers.PointMagneticFluxDensity(
        receiver_location, times, orientation=receiver_orientation
    )
)

# Define the source waveform.
#https://docs.simpeg.xyz/latest/content/api/generated/simpeg.electromagnetics.time_domain.sources.StepOffWaveform.html#simpeg.electromagnetics.time_domain.sources.StepOffWaveform
waveform = tdem.sources.StepOffWaveform()

# Sources
#https://docs.simpeg.xyz/latest/content/api/generated/simpeg.electromagnetics.time_domain.sources.CircularLoop.html#simpeg.electromagnetics.time_domain.sources.CircularLoop
source_list = [
    tdem.sources.CircularLoop(
        receiver_list=receiver_list,
        location=source_location,
        waveform=waveform,
        current=source_current,
        radius=source_radius,
    )
]

# Survey
survey = tdem.Survey(source_list)

In [ ]:
# 5% of the absolute value
uncertainties = 0.05 * np.abs(dobs) * np.ones(np.shape(dobs))

# Define the data object
data_object = data.Data(survey, dobs=dobs, standard_deviation=uncertainties)

# Create Inversion Mesh

In [ ]:
# Layer thicknesses
inv_thicknesses = np.logspace(0, 1.5, 25)
print((np.r_[inv_thicknesses, inv_thicknesses[-1]]))
# Define a mesh for plotting and regularization.
#https://discretize.simpeg.xyz/en/latest/api/generated/discretize.TensorMesh.html
mesh = TensorMesh([(np.r_[inv_thicknesses, inv_thicknesses[-1]])], "0")
print(mesh)

# Define model

In [ ]:
# Define model. A resistivity (Ohm meters) or conductivity (S/m) for each layer.
starting_model = np.log(0.1 * np.ones(mesh.nC))

# Define mapping from model to active cells.
model_mapping = maps.ExpMap()

# Create forward model (i.e., simulation)

In [ ]:
#https://docs.simpeg.xyz/latest/content/api/generated/simpeg.electromagnetics.time_domain.Simulation1DLayered.html#simpeg.electromagnetics.time_domain.Simulation1DLayered
simulation = tdem.Simulation1DLayered(
    survey=survey, thicknesses=inv_thicknesses, sigmaMap=model_mapping
)

# Set up model parameters

In [ ]:
# Define the data misfit. Here the data misfit is the L2 norm of the weighted
# residual between the observed data and the data predicted for a given model.
# The weighting is defined by the reciprocal of the uncertainties.
dmis = data_misfit.L2DataMisfit(simulation=simulation, data=data_object)
dmis.W = 1.0 / uncertainties

# Define the regularization (model objective function)
reg_map = maps.IdentityMap(nP=mesh.nC)
reg = regularization.Sparse(mesh, mapping=reg_map, alpha_s=0.01, alpha_x=1.0)

# set reference model
reg.reference_model = starting_model

# Define sparse and blocky norms p, q
reg.norms = [1, 0]

# Define how the optimization problem is solved. Here we will use an inexact
# Gauss-Newton approach that employs the conjugate gradient solver.
opt = optimization.ProjectedGNCG(maxIter=100, maxIterLS=20, maxIterCG=30, tolCG=1e-3)

# Define the inverse problem
inv_prob = inverse_problem.BaseInvProblem(dmis, reg, opt)

In [ ]:
# Defining a starting value for the trade-off parameter (beta) between the data
# misfit and the regularization.
starting_beta = directives.BetaEstimate_ByEig(beta0_ratio=1e2)

# Update the preconditionner
update_Jacobi = directives.UpdatePreconditioner()

# Options for outputting recovered models and predicted data for each beta.
save_iteration = directives.SaveOutputEveryIteration(save_txt=False)

# Directives for the IRLS
update_IRLS = directives.UpdateIRLS(max_irls_iterations=30, irls_cooling_factor=1.5)

# Updating the preconditionner if it is model dependent.
update_jacobi = directives.UpdatePreconditioner()

# Add sensitivity weights
sensitivity_weights = directives.UpdateSensitivityWeights()

# The directives are defined as a list.
directives_list = [
    sensitivity_weights,
    starting_beta,
    save_iteration,
    update_IRLS,
    update_jacobi,
]

In [ ]:
# Here we combine the inverse problem and the set of directives
inv = inversion.BaseInversion(inv_prob, directives_list)

import warnings
import simpeg
warnings.filterwarnings('ignore', category=simpeg.utils.solver_utils.DefaultSolverWarning)

# Run the inversion
recovered_model = inv.run(starting_model)

In [ ]:
# Load the true model and layer thicknesses
true_model = np.array([0.1, 1.0, 0.1])
true_layers = np.r_[40.0, 40.0, 160.0]

# Extract Least-Squares model
l2_model = inv_prob.l2model
print(np.shape(l2_model))

# Plot true model and recovered model
fig = plt.figure(figsize=(8, 9))
x_min = np.min(
    np.r_[model_mapping * recovered_model, model_mapping * l2_model, true_model]
)
x_max = np.max(
    np.r_[model_mapping * recovered_model, model_mapping * l2_model, true_model]
)

ax1 = fig.add_axes([0.2, 0.15, 0.7, 0.7])
plot_1d_layer_model(true_layers, true_model, ax=ax1, show_layers=False, color="k")

# "Smooth (L2) model"
plot_1d_layer_model(
    mesh.h[0], model_mapping * l2_model, ax=ax1, show_layers=False, color="b"
)

# "Blocky" (L0/L1) model
plot_1d_layer_model(
    mesh.h[0], model_mapping * recovered_model, ax=ax1, show_layers=False, color="r"
)
ax1.set_xlim(0.01, 10)
ax1.grid()
ax1.set_title("True and Recovered Models")
ax1.legend(["True Model", "L2-Model", "Sparse Model"])
#plt.gca().invert_yaxis()

# Plot predicted and observed data
dpred_l2 = simulation.dpred(l2_model)
dpred_final = simulation.dpred(recovered_model)

fig = plt.figure(figsize=(7, 7))
ax1 = fig.add_axes([0.15, 0.15, 0.8, 0.75])
ax1.loglog(times, np.abs(dobs), "k-o")
ax1.loglog(times, np.abs(dpred_l2), "b-o")
ax1.loglog(times, np.abs(dpred_final), "r-o")
ax1.grid()
ax1.set_xlabel("times (Hz)")
ax1.set_ylabel("|Hs/Hp| (ppm)")
ax1.set_title("Predicted and Observed Data")
ax1.legend(["Observed", "L2-Model", "Sparse"], loc="upper right")
plt.show()